In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import ConvLSTM2D, LSTM, BatchNormalization, Conv2D,Input, Dense, Reshape, Concatenate
from keras.layers import Masking, InputLayer
from utils.loss import masked_mse
import numpy as np
from utils.dataloader import dataloader
from keras.utils import plot_model
from utils import get_config
from keras.layers import ConvLSTM2D, LSTM, BatchNormalization, Conv2D,Input, Dense, Reshape, Concatenate
from keras.layers import Masking, Embedding
from keras.layers import LayerNormalization, MultiHeadAttention, Add

In [2]:

class ConvLSTM:

    def __init__(self, x_iv_train, y_iv_train, \
                 x_iv_val=None, y_iv_val=None, config=None):
        self.read_config(config) # Read the parameters and set the data
        self.x_iv_train = x_iv_train
        self.target_train = y_iv_train
        self.x_iv_val = x_iv_val
        self.target_val = y_iv_val

    def read_config(self, config):

        self.patience = config['training']['patience']
        self.epsilon = config['training']['epsilon']
        self.batch_size = config['training']['batch_size']
        self.epochs = config['training']['epochs']
        self.seed = config['training']['seed']
        self.learning_rate = config['training']['lr']

        self.window_size = config['data']['window_size']
        self.run = config['data']['run']
        self.covariate_columns = config['data']['covariates']
        self.option_type = config['data']['option']
        self.smooth = config['data']['smooth']
        self.h_step = config['data']['h_step']

        self.filters = config['model']['filters'] # 16 32 64 128 filter within the conv2DLSTM layer
        self.kernel_height = config['model']['kernel_height']
        self.kernel_width = config['model']['kernel_width'] # 1 to 5 (maturity) mxn -> 9x5
        self.num_layer = config['model']['num_layer'] # Any positive integer >0
        self.strides_dim = config['model']['strides_dim'] #: !!int 1 # assumes strides to be same across the two dimensions 
        self.kernel_initializer = config['model']['kernel_initializer'] 
        self.recurrent_initializer = config['model']['recurrent_initializer']
        self.padding = config['model']['padding']
        self.conv_activation = config['model']['conv_activation']
        self.recurrent_activation = config['model']['recurrent_activation']

    def compile(self):
        # set seed before compiling
        tf.random.set_seed(self.seed)

        time_steps = self.window_size
        _, window, height, width, _ = self.x_iv_train.shape
        channels = 1 
        # height = len(data_train['moneyness'].unique())
        # width = len(data_train['maturity'].unique())
        self.model = Sequential()
        self.model.add(InputLayer(input_shape=(time_steps, height, width, channels)))
        self.model.add(Masking(mask_value=0.0))
        # self.model.add(Masking(mask_value=0.0))
        # ConvLSTM2D expects 5D input: (batch, time, height, width, channels)

        for i in range(self.num_layer-1):
            self.model.add(ConvLSTM2D(filters=self.filters, 
                                      kernel_size=(self.kernel_height, self.kernel_width),
                                      strides=(self.strides_dim, self.strides_dim),
                                    padding=self.padding, 
                                    return_sequences=True,
                                    kernel_initializer=self.kernel_initializer,
                                    recurrent_initializer=self.recurrent_initializer,
                                    activation=self.conv_activation,
                                    recurrent_activation=self.recurrent_activation
                                ))
            self.model.add(BatchNormalization())

        self.model.add(ConvLSTM2D(filters=self.filters, 
                                  kernel_size=(self.kernel_height, self.kernel_width),
                                  strides=(self.strides_dim, self.strides_dim),
                                  padding=self.padding, 
                                  return_sequences=False,
                                  kernel_initializer=self.kernel_initializer,
                                  recurrent_initializer=self.recurrent_initializer,
                                  activation=self.conv_activation,
                                  recurrent_activation=self.recurrent_activation))
        self.model.add(BatchNormalization())

        # Final 3D convolution to map to the next frame
        self.model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1),
                                        activation='sigmoid', padding='same'))

        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, epsilon=self.epsilon)
        self.model.compile(loss=masked_mse, optimizer=self.optimizer)

        # Double check the architecture, and the activaiton function
        print(self.model.summary())

    def fit(self):
        self.history = self.model.fit(self.x_iv_train, self.target_train,
                validation_data=(self.x_iv_val, self.target_val),
                epochs=self.epochs, batch_size=self.batch_size, shuffle=False,
                callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                            patience=self.patience,
                                                            mode='min')])
        best_epoch = int(np.argmin(self.history.history['val_loss'])) + 1
        best_val_loss = self.history.history['val_loss'][best_epoch-1]
        train_loss = self.history.history['loss']
        val_loss = self.history.history.get('val_loss')
        return best_epoch, best_val_loss, train_loss, val_loss
    
    def fit_test(self, num_epoch):
        self.model.fit(self.x_iv_train, self.target_train,
                epochs=num_epoch, batch_size=self.batch_size, shuffle=False)
    
    def pred(self, x_iv): 
        pred = self.model.predict(x_iv)
        return pred
    
    def plot_architecture(self, filename='convlstm.png'):
        plot_model(self.model, to_file=filename, show_shapes=True, show_layer_names=False,
                   dpi=300, rankdir='TB')
    

In [3]:
run = 'short_ttm'
option_type ='put'
smooth=True
full_train=False
covariate_columns = []
h_step = 1
temporary_map = 'data/final/binned'
window_size = 21

In [4]:
x_iv_train, x_cov_train, target_train, x_iv_val, x_cov_val, \
            target_val, x_iv_test, x_cov_test, target_test, IV_train, IV_val, IV_test = \
                dataloader(run, option_type, smooth, full_train, covariate_columns, window_size, 
                           h_step, folder_path=temporary_map)

In [ ]:
config_original = 'config_file.yaml'
config = get_config(config_original)

In [6]:
model = ConvLSTM(x_iv_train, target_train, x_iv_val, target_val, config)

In [7]:
model.compile()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 21, 9, 5, 1)       0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 21, 9, 5, 64)      150016    
                                                                 
 batch_normalization (BatchN  (None, 21, 9, 5, 64)     256       
 ormalization)                                                   
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 9, 5, 64)          295168    
                                                                 
 batch_normalization_1 (Batc  (None, 9, 5, 64)         256       
 hNormalization)                                                 
                                                                 
 conv2d (Conv2D)             (None, 9, 5, 1)           6

In [8]:
model.plot_architecture('figures/architectures/convlstm.png')

In [9]:

class CovConvLSTM:

    def __init__(self, x_iv_train, x_cov_train, y_iv_train, \
                 x_iv_val=None, x_cov_val=None, y_iv_val=None, config=None):
        self.read_config(config) # Read the parameters and set the data
        self.x_iv_train = x_iv_train
        self.x_cov_train = x_cov_train
        self.target_train = y_iv_train
        self.x_iv_val = x_iv_val
        self.x_cov_val = x_cov_val
        self.target_val = y_iv_val

    def read_config(self, config):
       
        self.patience = config['training']['patience']
        self.epsilon = config['training']['epsilon']
        self.batch_size = config['training']['batch_size']
        self.epochs = config['training']['epochs']
        self.seed = config['training']['seed']
        self.learning_rate = config['training']['lr']

        self.window_size = config['data']['window_size']
        self.run = config['data']['run']
        self.covariate_columns = config['data']['covariates']
        self.option_type = config['data']['option']
        self.smooth = config['data']['smooth']
        self.h_step = config['data']['h_step']

        self.filters = config['model']['filters'] # 16 32 64 128 filter within the conv2DLSTM layer
        self.kernel_height = config['model']['kernel_height']
        self.kernel_width = config['model']['kernel_width'] # 1 to 5 (maturity) mxn -> 9x5
        self.num_layer = config['model']['num_layer'] # Any positive integer >0
        self.strides_dim = config['model']['strides_dim'] #: !!int 1 # assumes strides to be same across the two dimensions 
        self.kernel_initializer = config['model']['kernel_initializer'] 
        self.recurrent_initializer = config['model']['recurrent_initializer']
        self.padding = config['model']['padding']
        self.conv_activation = config['model']['conv_activation']
        self.recurrent_activation = config['model']['recurrent_activation']

    def compile(self):
        # set seed before compiling
        tf.random.set_seed(self.seed)

        time_steps = self.window_size
        _, window, height, width, _ = self.x_iv_train.shape
        # height = len(data_train['moneyness'].unique())
        # width = len(data_train['maturity'].unique())
        num_covariates = len(self.covariate_columns)

        # skip seed for now, first check results
        
        iv_input = Input(shape=(time_steps, height, width, 1), name="iv_input")
        x_iv = Masking(mask_value=0.0)(iv_input)

        for i in range(self.num_layer-1):
            x_iv = ConvLSTM2D(filters=self.filters, 
                                    kernel_size=(self.kernel_height, self.kernel_width),
                                    strides=(self.strides_dim, self.strides_dim),
                                    padding=self.padding, 
                                    return_sequences=True,
                                    kernel_initializer=self.kernel_initializer,
                                    recurrent_initializer=self.recurrent_initializer,
                                    activation=self.conv_activation,
                                    recurrent_activation=self.recurrent_activation)(iv_input)
            x_iv = BatchNormalization()(x_iv)

        x_iv = ConvLSTM2D(filters=self.filters, 
                                    kernel_size=(self.kernel_height, self.kernel_width),
                                    strides=(self.strides_dim, self.strides_dim),
                                    padding=self.padding, 
                                    return_sequences=False,
                                    kernel_initializer=self.kernel_initializer,
                                    recurrent_initializer=self.recurrent_initializer,
                                    activation=self.conv_activation,
                                    recurrent_activation=self.recurrent_activation)(x_iv)
        x_iv = BatchNormalization()(x_iv)

        cov_input = Input(shape=(time_steps, num_covariates), name="cov_input")
        # LSTM layer for the covariates
        x_cov = LSTM(units=64, return_sequences=False)(cov_input)   # (batch_size, 64)
        x_cov = Dense(units=height * width, activation='relu')(x_cov)  
        x_cov = Reshape((height, width, 1))(x_cov)               # (batch_size, H, W, 1)

        x = Concatenate(axis=-1)([x_iv, x_cov])  # Combine along channel axis -> (H, W, 65)
        x = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid', padding='same')(x)

        self.model = Model(inputs=[iv_input, cov_input], outputs=x)

        optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, epsilon=self.epsilon)
        self.model.compile(loss=masked_mse, optimizer=optimizer)
        print(self.model.summary())

    def fit(self):
        self.history = self.model.fit([self.x_iv_train, self.x_cov_train], self.target_train,
                validation_data=([self.x_iv_val, self.x_cov_val], self.target_val),
                epochs=self.epochs, batch_size=self.batch_size, shuffle=False,
                callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                            patience=self.patience,
                                                            mode='min')])
        best_epoch = int(np.argmin(self.history.history['val_loss'])) + 1
        best_val_loss = self.history.history['val_loss'][best_epoch]
        train_loss = self.history.history['loss']
        val_loss = self.history.history.get('val_loss')
        return best_epoch, best_val_loss, train_loss, val_loss
    
    def fit_test(self, num_epoch):
        self.model.fit([self.x_iv_train, self.x_cov_train], self.target_train,
                epochs=num_epoch, batch_size=self.batch_size, shuffle=False)
    
    def pred(self, x_iv, x_cov): 
        pred = self.model.predict([x_iv, x_cov])
        return pred
    
    def plot_architecture(self, filename='covconvlstm.png'):
        plot_model(self.model, to_file=filename, show_shapes=True, show_layer_names=False,
                   dpi=300, rankdir='TB')
    

In [10]:
config_name = 'config_file_covs2.yaml'
config = get_config(config_name)

covariate_columns = config['data']['covariates']

In [11]:
x_iv_train, x_cov_train, target_train, x_iv_val, x_cov_val, \
            target_val, x_iv_test, x_cov_test, target_test, IV_train, IV_val, IV_test = \
                dataloader(run, option_type, smooth, full_train, covariate_columns, window_size, 
                           h_step, folder_path=temporary_map)

In [12]:
conv_lstm = CovConvLSTM(x_iv_train, x_cov_train, target_train, x_iv_val, x_cov_val, target_val, config)

In [13]:
conv_lstm.compile()
conv_lstm.plot_architecture('figures/architectures/covconvlstm.png')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 iv_input (InputLayer)          [(None, 21, 9, 5, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv_lstm2d_2 (ConvLSTM2D)     (None, 21, 9, 5, 64  199936      ['iv_input[0][0]']               
                                )                                                                 
                                                                                                  
 cov_input (InputLayer)         [(None, 21, 8)]      0           []                               
                                                                                              

In [14]:

class Transformer:

    def __init__(self, x_iv_train, y_iv_train, \
                 x_iv_val=None, y_iv_val=None, config=None):
        self.read_config(config) 
        self.x_iv_train = x_iv_train
        self.target_train = y_iv_train
        self.x_iv_val = x_iv_val
        self.target_val = y_iv_val

    def read_config(self, config):

        self.patience = config['training']['patience']
        self.epsilon = config['training']['epsilon']
        self.batch_size = config['training']['batch_size']
        self.epochs = config['training']['epochs']
        self.seed = config['training']['seed']
        self.learning_rate = config['training']['lr']

        self.window_size = config['data']['window_size']
        self.run = config['data']['run']
        self.covariate_columns = config['data']['covariates']
        self.option_type = config['data']['option']
        self.smooth = config['data']['smooth']
        self.h_step = config['data']['h_step']

        # self.filters = config['model']['filters'] # 16 32 64 128 filter within the conv2DLSTM layer
        # self.kernel_height = config['model']['kernel_height']
        # self.kernel_width = config['model']['kernel_width'] # 1 to 5 (maturity) mxn -> 9x5
        # self.num_layer = config['model']['num_layer'] # Any positive integer >0
        # self.strides_dim = config['model']['strides_dim'] #: !!int 1 # assumes strides to be same across the two dimensions 
        # self.kernel_initializer = config['model']['kernel_initializer'] 
        # self.recurrent_initializer = config['model']['recurrent_initializer']
        # self.padding = config['model']['padding']
        # self.conv_activation = config['model']['conv_activation']
        # self.recurrent_activation = config['model']['recurrent_activation']
        self.num_heads = config['model']['num_heads']
        self.key_dim = config['model']['key_dim']

    def compile(self):
        tf.random.set_seed(self.seed)

        time_steps = self.window_size
        _, window, height, width, _ = self.x_iv_train.shape
        channels = 1

        inputs = tf.keras.Input(shape=(time_steps, height, width, channels))
        x = tf.keras.layers.Reshape((time_steps, height * width))(inputs)  # flatten spatial grid
        x = tf.keras.layers.Dense(64)(x)  # project to embedding dim

        def create_causal_mask(seq_len):
            return np.triu(np.ones((seq_len, seq_len)), k=1)

        # When calling the self-attention layer
        mask = create_causal_mask(time_steps)  # time_steps is the sequence length

        self_attention = tf.keras.layers.MultiHeadAttention(
            num_heads=self.num_heads,
            key_dim=self.key_dim,
            name="self_attention",
        )

        attention_output, attention_scores = self_attention(
            x, x, attention_mask=mask, return_attention_scores=True
        )


        x = tf.keras.layers.Add()([x, attention_output])
        x = tf.keras.layers.LayerNormalization()(x)
        x = tf.keras.layers.Dense(64, activation='relu')(x)
        x = tf.keras.layers.Dense(height * width)(x)
        outputs = tf.keras.layers.Reshape((height, width, 1))(x[:, -1])  # predict for last time step

        self.model = tf.keras.Model(inputs=inputs, outputs=outputs)

        self.optimizer = tf.keras.optimizers.Adam(
            learning_rate=self.learning_rate, epsilon=self.epsilon)
        self.model.compile(loss=masked_mse, optimizer=self.optimizer)

        print(self.model.summary())


    def fit(self):
        self.history = self.model.fit(self.x_iv_train, self.target_train,
                validation_data=(self.x_iv_val, self.target_val),
                epochs=self.epochs, batch_size=self.batch_size, shuffle=False,
                callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                            patience=self.patience,
                                                            mode='min')])
        best_epoch = int(np.argmin(self.history.history['val_loss'])) + 1
        best_val_loss = self.history.history['val_loss'][best_epoch-1]
        train_loss = self.history.history['loss']
        val_loss = self.history.history.get('val_loss')
        return best_epoch, best_val_loss, train_loss, val_loss
    
    def fit_test(self, num_epoch):
        self.model.fit(self.x_iv_train, self.target_train,
                epochs=num_epoch, batch_size=self.batch_size, shuffle=False)
    

    def get_attention(self):
        sample_input = self.x_iv_val[0:1]  #(1, time, height, width, channels)

        flattened = tf.reshape(sample_input, (1, sample_input.shape[1], -1))  # (1, time, H*W)

        dense = tf.keras.layers.Dense(64)  
        embedded = dense(flattened)

        # attention layer
        attention_layer = self.model.get_layer("self_attention")
        attention_output, attn_weights = attention_layer(
            embedded, embedded, return_attention_scores=True)
        
        return attn_weights
    
    def pred(self, x_iv): 
        pred = self.model.predict(x_iv)
        return pred
    
    def plot_architecture(self, filename='transformer.png'):
        plot_model(self.model, to_file=filename, show_shapes=True, show_layer_names=False,
                   dpi=300, rankdir='TB')
    

In [15]:
config_original = 'config_file_transformer.yaml'
config = get_config(config_original)

In [16]:
transformer = Transformer(x_iv_train, target_train, x_iv_val, target_val, config)
transformer.compile()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 21, 9, 5, 1  0           []                               
                                )]                                                                
                                                                                                  
 reshape_1 (Reshape)            (None, 21, 45)       0           ['input_2[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 21, 64)       2944        ['reshape_1[0][0]']              
                                                                                                  
 self_attention (MultiHeadAtten  ((None, 21, 64),    16640       ['dense_1[0][0]',          

In [17]:
transformer.plot_architecture('figures/architectures/transformer.png')

In [18]:
class CovTransformer:

    def __init__(self, x_iv_train, x_cov_train, y_iv_train, \
                 x_iv_val=None, x_cov_val=None, y_iv_val=None, config=None):
        self.read_config(config) 
        self.x_iv_train = x_iv_train
        self.x_cov_train = x_cov_train
        self.target_train = y_iv_train
        self.x_iv_val = x_iv_val
        self.x_cov_val = x_cov_val
        self.target_val = y_iv_val

    def read_config(self, config):
       
        self.patience = config['training']['patience']
        self.epsilon = config['training']['epsilon']
        self.batch_size = config['training']['batch_size']
        self.epochs = config['training']['epochs']
        self.seed = config['training']['seed']
        self.learning_rate = config['training']['lr']

        self.window_size = config['data']['window_size']
        self.run = config['data']['run']
        self.covariate_columns = config['data']['covariates']
        self.option_type = config['data']['option']
        self.smooth = config['data']['smooth']
        self.h_step = config['data']['h_step']

        self.num_heads = config['model']['num_heads']
        self.key_dim = config['model']['key_dim']

    def compile(self):
        tf.random.set_seed(self.seed)

        time_steps = self.window_size
        _, window, height, width, _ = self.x_iv_train.shape
        num_covariates = len(self.covariate_columns)
        patch_dim = height * width

        iv_input = Input(shape=(time_steps, height, width, 1), name="iv_input")
        cov_input = Input(shape=(time_steps, num_covariates), name="cov_input")
        
        x_iv = tf.reshape(iv_input, [-1, time_steps, patch_dim])  # (B, T, H*W)
        x_iv = Dense(64)(x_iv)  # feature space
        x_iv = LayerNormalization()(x_iv)

        positions = tf.range(start=0, limit=time_steps, delta=1)
        pos_encoding = Embedding(input_dim=time_steps, output_dim=64)(positions) 
        x_iv += pos_encoding  

        def create_causal_mask(seq_len):
           return np.triu(np.ones((seq_len, seq_len)), k=1)

        # # When calling the self-attention layer
        mask = create_causal_mask(time_steps)  # time_steps is the sequence length

        # self_attention = tf.keras.layers.MultiHeadAttention(
        #     num_heads=self.num_heads,
        #     key_dim=self.key_dim,
        #     name="self_attention",
        # )

        # attention_output, attention_scores = self_attention(
        #     x, x, attention_mask=mask, return_attention_scores=True
        # )

        #Transformer 
        attn_output = MultiHeadAttention(num_heads=self.num_heads, 
                                         key_dim=self.key_dim)(x_iv, x_iv, attention_mask = mask) 
        x = Add()([x_iv, attn_output])
        x = LayerNormalization()(x)
        #Feedforward layer
        ff = Dense(128, activation='relu')(x)
        ff = Dense(64)(ff)
        x = Add()([x, ff])
        x = LayerNormalization()(x)

        x_iv_out = x[:, -1, :]  
        x_cov = LSTM(units=64, return_sequences=False)(cov_input)
        x_cov = Dense(units=64, activation='relu')(x_cov)

        # concat outputs
        x = Concatenate()([x_iv_out, x_cov])
        x = Dense(units=patch_dim, activation='relu')(x)
        x = Reshape((height, width, 1))(x)

        x = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid', padding='same')(x)

        self.model = Model(inputs=[iv_input, cov_input], outputs=x)
        optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, epsilon=self.epsilon)
        self.model.compile(loss=masked_mse, optimizer=optimizer)
        print(self.model.summary())


    def fit(self):
        self.history = self.model.fit([self.x_iv_train, self.x_cov_train], self.target_train,
                validation_data=([self.x_iv_val, self.x_cov_val], self.target_val),
                epochs=self.epochs, batch_size=self.batch_size, shuffle=False,
                callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                            patience=self.patience,
                                                            mode='min')])
        best_epoch = int(np.argmin(self.history.history['val_loss'])) + 1
        best_val_loss = self.history.history['val_loss'][best_epoch]
        train_loss = self.history.history['loss']
        val_loss = self.history.history.get('val_loss')
        return best_epoch, best_val_loss, train_loss, val_loss
    
    def fit_test(self, num_epoch):
        self.model.fit([self.x_iv_train, self.x_cov_train], self.target_train,
                epochs=num_epoch, batch_size=self.batch_size, shuffle=False)
    
    def pred(self, x_iv, x_cov): 
        pred = self.model.predict([x_iv, x_cov])
        return pred
    
    def plot_architecture(self, filename='covtransformer.png'):
        plot_model(self.model, to_file=filename, show_shapes=True, show_layer_names=False,
                   dpi=300, rankdir='TB')
    

In [19]:
config_original = 'config_file_covtransformer.yaml'
config = get_config(config_original)


transformer = CovTransformer(x_iv_train, x_cov_train, target_train, x_iv_val, x_cov_val, target_val, config)
transformer.compile()
transformer.plot_architecture("figures/architectures/covtransformer.png")

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 iv_input (InputLayer)          [(None, 21, 9, 5, 1  0           []                               
                                )]                                                                
                                                                                                  
 tf.reshape (TFOpLambda)        (None, 21, 45)       0           ['iv_input[0][0]']               
                                                                                                  
 dense_4 (Dense)                (None, 21, 64)       2944        ['tf.reshape[0][0]']             
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 21, 64)      128         ['dense_4[0][0]']          